In [1]:
import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
df_nutrients = pd.read_csv("rdi.csv")

In [3]:
df_nutrients

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),...,Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
0,2500,96.0,281.25,97.222222,30.0,11.111111,1.388889,0.125,0.125,1000.0,...,70.0,110.0,1.0467,1.6,16.7472,5.0,1.7,330.0,4.0,750.0


In [4]:
nutrients = [(key,value) for key,value in df_nutrients.to_dict("records")[0].items()]
nutrients

[('Energy (kcal/d)', 2500),
 ('Protein (g/d)', 96.0),
 ('Carbohydrate (g/d)', 281.25),
 ('Fat (g/d)', 97.22222222222224),
 ('Fibres (g/d)', 30.0),
 ('LA (g/d)', 11.11111111111111),
 ('ALA (g/d)', 1.3888888888888888),
 ('EPA (g/d)', 0.125),
 ('DHA (g/d)', 0.125),
 ('Calcium (mg/d)', 1000.0),
 ('Chloride (mg/d)', 3100.0),
 ('Copper (mg/d)', 1.6),
 ('Iron (mg/d)', 11.0),
 ('Iodine (µg/d)', 150.0),
 ('Magnesium (mg/d)', 350.0),
 ('Manganese (mg/d)', 3.0),
 ('Phosphorus (mg/d)', 550.0),
 ('Potassium (mg/d)', 3500.0),
 ('Selenium (µg/d)', 70.0),
 ('Sodium (mg/d)', 2000.0),
 ('Zinc (mg/d)', 10.0),
 ('Vitamin D (µg/d)', 15.0),
 ('Vitamin E (mg/d)', 13.0),
 ('Vitamin K1 (µg/d)', 70.0),
 ('Vitamin C (mg/d)', 110.0),
 ('Vitamin B1 or Thiamin (mg/d)', 1.0467000000000002),
 ('Vitamin B2 or Riboflavin (mg/d)', 1.6),
 ('Vitamin B3 or Niacin (mg/d)', 16.747200000000003),
 ('Vitamin B5 or Pantothenic acid (mg/d)', 5.0),
 ('Vitamin B6 (mg/d)', 1.7),
 ('Vitamin B9 or Folate (µg/d)', 330.0),
 ('Vitamin B1

In [5]:
df_foods=pd.read_csv("ciqual_2020.csv")

In [6]:
# https://stackoverflow.com/questions/11350770/filter-pandas-dataframe-by-substring-criteria
def filter_rows_by_substrings(df, col, substrings):
    return df[~df[col].str.contains('|'.join(substrings))]


# https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [7]:
# remove certain foods groups, subgroups and subsubgroups

remove_groups = ["baby food", "sugar and confectionery", "beverages", "fats and oils"]

df_foods_f1 = filter_rows_by_values(df_foods, "Group", remove_groups)

remove_subgroups = ["savoury biscuits", "breads and similar", "spices", "sauces", "herbs", "cheese and similar"]

df_foods_f2 = filter_rows_by_values(df_foods_f1, "Subgroup", remove_subgroups)

remove_subsubgroups = []

df_foods_f3 = filter_rows_by_values(df_foods_f2, "Subsubgroup", remove_subsubgroups)

In [8]:
substrings = ["crisps", 
              "powder", 
              "flour", 
              "dried", 
              "dehydrated", 
              "Shortbread", 
              "pastry", "Pastry", 
              "Almond paste or marzipan. prepacked", 
              "Nutritional yeast", 
              "Royal jelly", 
              "Poppy. seed", 
              "starch", 
              "Soy lecithin", 
              "Mix", "pre-cooked", "precooked", "Raisin"]

df_foods_filtered = filter_rows_by_substrings(df_foods_f3, "Name", substrings)

In [9]:
commodities = list(df_foods_filtered["Name"])

In [10]:
data = df_foods_filtered.drop(["Group", "Subgroup", "Subsubgroup", "Name"], axis=1).values.tolist()

In [11]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [12]:
# Declare an array to hold our variables. 
foods = [solver.NumVar(0.0, solver.infinity(), item) for item in commodities]

print('Number of variables =', solver.NumVariables())

Number of variables = 1950


In [13]:
# Create the constraints, one per nutrient. (data = nutrients_per_100_gramm)
# gurobipy can express a lists or arrays of constraints with a nicer DSL 
# instead of the many loops necessary with OR-Tools
constraints = []
for i, nutrient in enumerate(nutrients):
    constraints.append(solver.Constraint(nutrient[1], solver.infinity(), nutrient[0]))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 33


In [14]:
# Objective function: Minimize the sum of (price-normalized) foods.
objective = solver.Objective()
for i, food in enumerate(foods):
    objective.SetCoefficient(food, 1.0)
objective.SetMinimization()

In [15]:
status = solver.Solve()

# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)

In [16]:
# Display the amounts (in dollars) to purchase of each food.
print('\nDaily Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: {} gr'.format(commodities[i], food.solution_value()*100))
print('\nOptimal daily weight: {:.4f} gr'.format(objective.Value()*100))


Daily Foods:
Acerola. pulp. raw. sampled in the island of La Martinique: 3.395184446283294 gr
Cashew nut. dry-grilled. unsalted: 79.26107135571951 gr
Soybean. whole grain: 141.1641043302068 gr
Rice. parboiled. raw: 254.19879132988785 gr
Amaranth. raw: 56.34867648123254 gr
Cod liver. raw: 46.36169042358652 gr
Shrimp. cooked: 29.99592703597549 gr
Pure sea salt. no enrichment: 4.905935150825534 gr

Optimal daily weight: 615.6314 gr


In [17]:
food_per_day=[]

nutrients_result = [0] * len(nutrients)
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        # default value for weight optimization
        nutrient_per_goal = 100  
        food_per_day.append((commodities[i], food.solution_value() * nutrient_per_goal))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j] * food.solution_value()

df_food_per_day = pd.DataFrame(food_per_day, columns=("food", "weight"))
df_food_per_day.sort_values(by='weight', ascending=False)

,food,weight
3,Rice. parboiled. raw,254.198791
2,Soybean. whole grain,141.164104
1,Cashew nut. dry-grilled. unsalted,79.261071
4,Amaranth. raw,56.348676
5,Cod liver. raw,46.361690
6,Shrimp. cooked,29.995927
7,Pure sea salt. no enrichment,4.905935
0,Acerola. pulp. raw. sampled in the island of L...,3.395184


In [18]:
print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i],
                                       nutrient[1]))


Nutrients per day:
Energy (kcal/d): 2500.00 (min 2500)
Protein (g/d): 96.00 (min 96.0)
Carbohydrate (g/d): 281.25 (min 281.25)
Fat (g/d): 103.63 (min 97.22222222222224)
Fibres (g/d): 30.00 (min 30.0)
LA (g/d): 22.76 (min 11.11111111111111)
ALA (g/d): 2.31 (min 1.3888888888888888)
EPA (g/d): 3.06 (min 0.125)
DHA (g/d): 2.28 (min 0.125)
Calcium (mg/d): 1000.00 (min 1000.0)
Chloride (mg/d): 3100.00 (min 3100.0)
Copper (mg/d): 4.90 (min 1.6)
Iron (mg/d): 35.53 (min 11.0)
Iodine (µg/d): 316.24 (min 150.0)
Magnesium (mg/d): 838.03 (min 350.0)
Manganese (mg/d): 12.81 (min 3.0)
Phosphorus (mg/d): 2103.32 (min 550.0)
Potassium (mg/d): 3787.70 (min 3500.0)
Selenium (µg/d): 84.96 (min 70.0)
Sodium (mg/d): 2285.73 (min 2000.0)
Zinc (mg/d): 15.30 (min 10.0)
Vitamin D (µg/d): 46.63 (min 15.0)
Vitamin E (mg/d): 13.00 (min 13.0)
Vitamin K1 (µg/d): 91.65 (min 70.0)
Vitamin C (mg/d): 110.00 (min 110.0)
Vitamin B1 or Thiamin (mg/d): 2.70 (min 1.0467000000000002)
Vitamin B2 or Riboflavin (mg/d): 1.87 (mi

In [19]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food.name() in nutrient_per_food:
                nutrient_per_food[food.name()].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food.name()]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]/nutrients_result[i]*100).round(2)

display(foods_df)     

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Acerola. pulp. raw. sampled in the island of La Martinique,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,87.97,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cashew nut. dry-grilled. unsalted,19.56,14.37,6.62,37.48,15.06,31.06,3.09,0.26,0.35,3.01,0.61,43.70,12.72,0.00,26.48,11.13,19.97,14.23,18.66,0.28,29.02,0.42,3.54,27.07,0.36,10.57,2.07,2.56,13.26,6.06,2.85,0.00,0.05
Soybean. whole grain,23.66,50.73,10.44,26.15,61.17,56.06,78.92,0.00,0.00,31.06,0.00,25.37,62.38,0.27,42.62,24.90,39.33,64.85,14.62,0.19,27.22,0.00,9.23,72.39,7.70,45.49,65.52,13.21,32.78,25.40,65.06,0.00,0.13
Rice. parboiled. raw,35.99,18.83,71.04,2.40,11.18,3.80,0.98,0.41,0.56,25.67,0.00,12.46,6.94,1.77,9.40,49.00,19.58,10.87,17.95,0.56,22.60,0.00,0.78,0.28,0.00,39.55,5.83,73.69,34.29,49.18,4.64,0.00,0.00
Amaranth. raw,8.23,7.92,11.74,3.82,12.58,6.78,1.00,0.00,0.00,8.96,0.00,6.10,12.07,0.00,16.68,14.64,14.92,7.56,0.00,0.10,10.57,0.00,5.16,0.00,2.15,2.50,6.01,2.99,14.05,14.96,6.49,0.00,0.00
Cod liver. raw,11.48,2.41,0.00,29.80,0.00,2.28,15.87,96.98,96.86,0.46,0.00,6.25,5.22,73.30,0.44,0.00,2.20,1.59,34.65,11.95,5.88,99.41,71.33,0.00,1.69,1.72,16.08,6.69,5.07,3.13,19.54,55.09,99.55
Shrimp. cooked,1.08,5.72,0.16,0.35,0.00,0.01,0.13,2.35,2.24,30.00,6.49,6.13,0.67,24.66,1.90,0.33,3.99,0.77,14.12,6.01,4.71,0.16,9.97,0.26,0.14,0.17,4.48,0.85,0.56,1.27,1.41,44.91,0.27
Pure sea salt. no enrichment,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.84,92.90,0.00,0.00,0.00,2.48,0.00,0.00,0.13,0.00,80.92,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [20]:
foods_df.sum()

Energy (kcal/d)                          100.00
Protein (g/d)                            100.00
Carbohydrate (g/d)                       100.00
Fat (g/d)                                100.00
Fibres (g/d)                              99.99
LA (g/d)                                  99.99
ALA (g/d)                                 99.99
EPA (g/d)                                100.00
DHA (g/d)                                100.01
Calcium (mg/d)                           100.00
Chloride (mg/d)                          100.00
Copper (mg/d)                            100.01
Iron (mg/d)                              100.00
Iodine (µg/d)                            100.00
Magnesium (mg/d)                         100.00
Manganese (mg/d)                         100.00
Phosphorus (mg/d)                         99.99
Potassium (mg/d)                         100.00
Selenium (µg/d)                          100.00
Sodium (mg/d)                            100.01
Zinc (mg/d)                             

In [21]:
import re
def remove_unit(column_name):
    return re.sub(r"\((.*)\)", "", column_name).strip()

In [22]:
for name in nutrient_per_food.keys():
    print(foods_df.loc[[name]].sort_values(name, axis=1, ascending=False).iloc[:, :5].to_dict("index"))

{'Acerola. pulp. raw. sampled in the island of La Martinique': {'Vitamin C (mg/d)': 87.97, 'Protein (g/d)': 0.02, 'Selenium (µg/d)': 0.0, 'Sodium (mg/d)': 0.0, 'Zinc (mg/d)': 0.0}}
{'Cashew nut. dry-grilled. unsalted': {'Copper (mg/d)': 43.7, 'Fat (g/d)': 37.48, 'LA (g/d)': 31.06, 'Zinc (mg/d)': 29.02, 'Vitamin K1 (µg/d)': 27.07}}
{'Soybean. whole grain': {'ALA (g/d)': 78.92, 'Vitamin K1 (µg/d)': 72.39, 'Vitamin B2 or Riboflavin (mg/d)': 65.52, 'Vitamin B9 or Folate (µg/d)': 65.06, 'Potassium (mg/d)': 64.85}}
{'Rice. parboiled. raw': {'Vitamin B3 or Niacin (mg/d)': 73.69, 'Carbohydrate (g/d)': 71.04, 'Vitamin B6 (mg/d)': 49.18, 'Manganese (mg/d)': 49.0, 'Vitamin B1 or Thiamin (mg/d)': 39.55}}
{'Amaranth. raw': {'Magnesium (mg/d)': 16.68, 'Vitamin B6 (mg/d)': 14.96, 'Phosphorus (mg/d)': 14.92, 'Manganese (mg/d)': 14.64, 'Vitamin B5 or Pantothenic acid (mg/d)': 14.05}}
{'Cod liver. raw': {'Vitamin A (µg/d)': 99.55, 'Vitamin D (µg/d)': 99.41, 'EPA (g/d)': 96.98, 'DHA (g/d)': 96.86, 'Iodin

In [23]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]).round(2)

# foods_df.loc['total'] = foods_df.iloc[1:, :-1].sum()    
    
display(foods_df)  

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Acerola. pulp. raw. sampled in the island of La Martinique,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,96.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cashew nut. dry-grilled. unsalted,489.04,13.79,18.63,38.84,4.52,7.07,0.07,0.01,0.01,30.12,18.94,2.14,4.52,0.00,221.93,1.43,420.08,538.98,15.85,6.34,4.44,0.20,0.46,24.81,0.40,0.29,0.04,0.44,0.78,0.13,20.29,0.00,1.23
Soybean. whole grain,591.48,48.70,29.36,27.10,18.35,12.76,1.82,0.00,0.00,310.56,0.00,1.24,22.16,0.85,357.15,3.19,827.22,2456.26,12.42,4.23,4.16,0.00,1.20,66.35,8.47,1.23,1.23,2.29,1.92,0.56,463.02,0.00,3.06
Rice. parboiled. raw,899.86,18.07,199.80,2.49,3.36,0.86,0.02,0.01,0.01,256.74,0.00,0.61,2.47,5.59,78.80,6.28,411.80,411.80,15.25,12.91,3.46,0.00,0.10,0.25,0.00,1.07,0.11,12.76,2.01,1.09,33.05,0.00,0.00
Amaranth. raw,205.67,7.61,33.02,3.96,3.78,1.54,0.02,0.00,0.00,89.59,0.00,0.30,4.29,0.00,139.74,1.88,313.86,286.25,0.00,2.25,1.62,0.00,0.67,0.00,2.37,0.07,0.11,0.52,0.82,0.33,46.21,0.00,0.09
Cod liver. raw,286.98,2.32,0.00,30.88,0.00,0.52,0.37,2.97,2.21,4.64,0.00,0.31,1.85,231.81,3.71,0.00,46.36,60.27,29.44,273.07,0.90,46.36,9.27,0.00,1.85,0.05,0.30,1.16,0.30,0.07,139.09,4.64,2364.45
Shrimp. cooked,26.97,5.49,0.44,0.36,0.00,0.00,0.00,0.07,0.05,299.96,201.27,0.30,0.24,77.99,15.90,0.04,83.99,29.10,12.00,137.38,0.72,0.07,1.30,0.24,0.15,0.00,0.08,0.15,0.03,0.03,10.05,3.78,6.30
Pure sea salt. no enrichment,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.39,2879.78,0.00,0.00,0.00,20.80,0.00,0.00,5.05,0.00,1849.54,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [24]:
foods_df.sum()

Energy (kcal/d)                          2500.00
Protein (g/d)                              96.00
Carbohydrate (g/d)                        281.25
Fat (g/d)                                 103.63
Fibres (g/d)                               30.01
LA (g/d)                                   22.75
ALA (g/d)                                   2.30
EPA (g/d)                                   3.06
DHA (g/d)                                   2.28
Calcium (mg/d)                           1000.00
Chloride (mg/d)                          3099.99
Copper (mg/d)                               4.90
Iron (mg/d)                                35.53
Iodine (µg/d)                             316.24
Magnesium (mg/d)                          838.03
Manganese (mg/d)                           12.82
Phosphorus (mg/d)                        2103.31
Potassium (mg/d)                         3787.71
Selenium (µg/d)                            84.96
Sodium (mg/d)                            2285.72
Zinc (mg/d)         

In [25]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack (%)': (activities[i] - c.lb())/c.lb()*100} for i, c in enumerate(solver.constraints())]
print(pd.DataFrame(o).round(2))

                                     Name  shadow price  slack (%)
0                         Energy (kcal/d)          0.00       0.00
1                           Protein (g/d)          0.00       0.00
2                      Carbohydrate (g/d)          0.01       0.00
3                               Fat (g/d)          0.00       6.59
4                            Fibres (g/d)          0.00      -0.00
5                                LA (g/d)          0.00     104.86
6                               ALA (g/d)          0.00      66.13
7                               EPA (g/d)          0.00    2351.53
8                               DHA (g/d)          0.00    1722.76
9                          Calcium (mg/d)          0.00       0.00
10                        Chloride (mg/d)          0.00       0.00
11                          Copper (mg/d)          0.00     206.06
12                            Iron (mg/d)          0.00     222.96
13                          Iodine (µg/d)          0.00     11

In [26]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack': (activities[i] - c.lb())} for i, c in enumerate(solver.constraints())]
df_sensitivity = pd.DataFrame(o)
print(df_sensitivity.round(2))

                                     Name  shadow price    slack
0                         Energy (kcal/d)          0.00     0.00
1                           Protein (g/d)          0.00     0.00
2                      Carbohydrate (g/d)          0.01     0.00
3                               Fat (g/d)          0.00     6.41
4                            Fibres (g/d)          0.00    -0.00
5                                LA (g/d)          0.00    11.65
6                               ALA (g/d)          0.00     0.92
7                               EPA (g/d)          0.00     2.94
8                               DHA (g/d)          0.00     2.15
9                          Calcium (mg/d)          0.00     0.00
10                        Chloride (mg/d)          0.00     0.00
11                          Copper (mg/d)          0.00     3.30
12                            Iron (mg/d)          0.00    24.53
13                          Iodine (µg/d)          0.00   166.24
14                       

The constraints with a slack value of zero are the most critical for the solution. 
It these constraints are changed the solution will also change. 
There are much more critical constraint for the calories optimized diet than for the weight optimized diet. 
The higher the shadow price the more sensitive is the objective function to changes of that constraint.
So the most critical constraints are 

In [27]:
df_sensitivity.sort_values("shadow price", ascending=False).loc[df_sensitivity["slack"].round(2) == 0.0].round(2)

,Name,shadow price,slack
22,Vitamin E (mg/d),0.01,0.0
2,Carbohydrate (g/d),0.01,0.0
1,Protein (g/d),0.00,0.0
4,Fibres (g/d),0.00,-0.0
0,Energy (kcal/d),0.00,0.0
9,Calcium (mg/d),0.00,0.0
24,Vitamin C (mg/d),0.00,0.0
10,Chloride (mg/d),0.00,0.0
